In [1]:
import requests
from bs4 import BeautifulSoup
import csv


In [2]:
# Make a request
page = requests.get("https://www.dsebd.org/company_listing.php")
soup = BeautifulSoup(page.content, 'html.parser')

In [3]:
# Create company_names as an empty list
company_names = []
# Find all the company names
companies = soup.find_all("div", {"class": "row al-li"})
for company in companies:
    company_name = company.text.strip()
    company_names.append(company_name)
print(company_names)

["A\n\nAAMRANET (aamra networks limited)\nAAMRATECH (aamra technologies limited)\nABB1STMF (AB Bank 1st Mutual fund)\nABBANK (AB Bank Limited)\nACFL (Aman Cotton Fibrous Limited)\nACI (ACI  Limited)\nACIFORMULA (ACI Formulations Limited)\nACMELAB (The ACME Laboratories Limited)\nACMEPL (ACME Pesticides Limited)\nACTIVEFINE (Active Fine Chemicals Limited)\nADNTEL (ADN Telecom Limited)\nADVENT (Advent Pharma Limited)\nAFCAGRO (AFC Agro Biotech Ltd.)\nAFTABAUTO (Aftab Automobiles Limited)\nAGNISYSL (Agni Systems Ltd.)\nAGRANINS (Agrani Insurance Co. Ltd.)\nAIBL1STIMF (AIBL 1st Islamic Mutual Fund)\nAIBLPBOND (AIBL Mudaraba Perpetual Bond)\nAIL (Alif Industries Limited)\nAL-HAJTEX (Al-Haj Textile Mills Limited)\nALARABANK (Al-Arafah Islami Bank Ltd)\nALIF (Alif Manufacturing Company Ltd.)\nALLTEX (Alltex Industries Ltd.)\nAMANFEED (Aman Feed Limited)\nAMBEEPHA (Ambee Pharmaceuticals  Ltd.)\nAMCL(PRAN) (Agricultural Marketing Company Ltd. (Pran))\nANLIMAYARN (Anlimayarn Dyeing Ltd.)\nANWARG

In [16]:
import csv
import requests
from bs4 import BeautifulSoup

page = requests.get("https://www.dsebd.org/company_listing.php")
soup = BeautifulSoup(page.content, 'html.parser')

# Create codes1 as an empty list
codes1 = []

# Find all the company names
trading = soup.find_all("div", {"class": "BodyContent"})
codes = soup.find_all("a", {"class": "ab1"})
for i in trading:
    code = i.find_all("a")
    for only_code in code:
        c = only_code.text.strip()
        codes1.append(c)
# Write the codes1 list to a CSV file
with open('trading.csv', 'w', newline='') as output_file:
    writer = csv.writer(output_file)
    for code in codes1:
        writer.writerow([code])
print()




In [25]:
import csv
import requests
from bs4 import BeautifulSoup

def company_info():
    # Read the trading names from the CSV file
    trading_names = []
    with open("trading.csv", "r") as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            trading_names.append(row[0])  # Assuming the trading names are in the first column

    # Create a list to store the company trading data
    company_trading = []

    # Traverse through the trading names
    for trading_name in trading_names:
        # Construct the URL for each company using the trading name
        url = f"https://www.dsebd.org/displayCompany.php?name={trading_name}"

        # Send a new HTTP GET request to the company's page
        page = requests.get(url)
        company_soup = BeautifulSoup(page.content, 'html.parser')

        # Find the company trading information
        trading_rows = company_soup.find_all("tr", class_="alt")
        for row in trading_rows:
            company_trade = row.text.strip()
            if company_trade.startswith("Trading Code:"):
                trading_info = company_trade.split("Scrip Code:")
                trading_code = trading_info[0].replace("Trading Code:", "").strip()
                scrip_code = trading_info[1].strip()

                # Find the company name
                company_name_h2 = company_soup.find('h2', {'class': 'BodyHead'})
                company_name = company_name_h2.text.strip()[company_name_h2.text.strip().find(": ") + len(": "):]

                # Find the table containing the sector information
                tag_value = company_soup.find_all("div", {"class": "table-responsive"})
                tr = tag_value[2].find_all("tr", {"class": "alt"})
                td = tr[1].find_all("td")
                sector = td[1].text.strip()

                company_trading.append((company_name, trading_code, scrip_code, sector))

    # Write to CSV file
    filename = "company_trading.csv"
    with open(filename, "w", newline="") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["Company Name", "Trading Code", "Scrip Code", "Sector"])  # Write header row
        writer.writerows(company_trading)  # Write data rows

    print("Data written to", filename)



def Holding_Percentage():
    otherinfo = []

    trading_names = []
    with open("trading.csv", "r") as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            trading_names.append(row[0])  # Assuming the trading names are in the first column

    # Traverse through the trading names
    for trading_name in trading_names:
        # Construct the URL for each company using the trading name
        url = f"https://www.dsebd.org/displayCompany.php?name={trading_name}"
        response = requests.get(url)
        html_content = response.content
        soup = BeautifulSoup(html_content, "html.parser")
        tag_value = soup.find_all("div", {"class": "table-responsive"})

        try:
            tr = tag_value[9].find_all("tr")

            if tr[3] in tr:
                tr1 = tr[3].find_all("td")
                date = tr1[0].text.strip()
                parsed_date = parse_date(date)
                float_numbers = re.findall(r'\d+\.\d+', tr1[1].text.strip())
                if len(float_numbers) == 5:
                    sponsor, govt, institute, foreign, public = float_numbers
                    otherinfo.append([trading_name, parsed_date, sponsor, govt, institute, foreign, public])

            # Similar logic for the other <tr> elements at indices 5 and 7

        except IndexError:
            print(f"No table found for {trading_name}")

    # Create DataFrame and save as CSV
    column_labels = ['Company_ID', 'Date', 'Sponsor', 'Govt', 'Institute', 'Foreign', 'Public']
    df = pd.DataFrame(otherinfo, columns=column_labels)
    df.to_csv('Holding_data.csv', index=False)
    print("Data written to Holding_data.csv")








Holding_Percentage()
company_info()


No table found for TB10Y0126
No table found for TB10Y0127
No table found for TB10Y0130
No table found for TB10Y0132
No table found for TB10Y0231
No table found for TB10Y0324
No table found for TB10Y0425
No table found for TB10Y0429
No table found for TB10Y0530
No table found for TB10Y0531
No table found for TB10Y0532
No table found for TB10Y0628
No table found for TB10Y0629
No table found for TB10Y0630
No table found for TB10Y0632
No table found for TB10Y0723
No table found for TB10Y0724
No table found for TB10Y0726
No table found for TB10Y0730
No table found for TB10Y0731
No table found for TB10Y0825
No table found for TB10Y0829
No table found for TB10Y0932
No table found for TB10Y1027
No table found for TB10Y1030
No table found for TB10Y1031
No table found for TB10Y1123
No table found for TB10Y1124
No table found for TB10Y1128
No table found for TB10Y1229
No table found for TB10Y1232
No table found for TB15Y0124
No table found for TB15Y0125
No table found for TB15Y0127
No table found

In [20]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd

otherinfo = []

trading_names = []
with open("trading.csv", "r") as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        trading_names.append(row[0])  # Assuming the trading names are in the first column

# Create a list to store the company trading data
company_trading = []

# Traverse through the trading names
for trading_name in trading_names:
    # Construct the URL for each company using the trading name
    url = f"https://www.dsebd.org/displayCompany.php?name={trading_name}"
    response = requests.get(url)
    html_content = response.content
    soup = BeautifulSoup(html_content, "html.parser")
    tag_value = soup.find_all("div", {"class": "table-responsive"})
    try:
    tr = tag_value[9].find_all("tr")

    try:
        if tr[3] in tr:
            tr1 = tr[3].find_all("td")
            date = tr1[0].text.strip()
            parsed_date = parse_date(date)
            float_numbers = re.findall(r'\d+\.\d+', tr1[1].text.strip())
            if len(float_numbers) == 5:
                sponsor, govt, institute, foreign, public = float_numbers
                otherinfo.append([trading_name, parsed_date, sponsor, govt, institute, foreign, public])

        # Similar logic for the other <tr> elements at indices 5 and 7

    except IndexError:
        print("Error occurred")

# Create DataFrame and save as CSV
column_labels = ['Company_ID', 'Date', 'Sponsor', 'Govt', 'Institute', 'Foreign', 'Public']
df = pd.DataFrame(otherinfo, columns=column_labels)
df.to_csv('Holding_data.csv', index=False)
print("Data written to Holding_data.csv")









No table found for TB10Y0126
No table found for TB10Y0127
No table found for TB10Y0130
No table found for TB10Y0132
No table found for TB10Y0231
No table found for TB10Y0324
No table found for TB10Y0425
No table found for TB10Y0429
No table found for TB10Y0530
No table found for TB10Y0531
No table found for TB10Y0532
No table found for TB10Y0628
No table found for TB10Y0629
No table found for TB10Y0630
No table found for TB10Y0632
No table found for TB10Y0723
No table found for TB10Y0724
No table found for TB10Y0726
No table found for TB10Y0730
No table found for TB10Y0731
No table found for TB10Y0825
No table found for TB10Y0829
No table found for TB10Y0932
No table found for TB10Y1027
No table found for TB10Y1030
No table found for TB10Y1031
No table found for TB10Y1123
No table found for TB10Y1124
No table found for TB10Y1128
No table found for TB10Y1229
No table found for TB10Y1232
No table found for TB15Y0124
No table found for TB15Y0125
No table found for TB15Y0127
No table found

In [19]:
import re
from datetime import datetime

def parse_date(text):
    # Use regular expressions to extract the date
    date_regex = r'\b\w{3} \d{2}, \d{4}\b'
    matches = re.findall(date_regex, text)

    if matches:
        date_str = matches[0]

        # Convert the extracted date string to datetime object
        date_obj = datetime.strptime(date_str, '%b %d, %Y')

        # Format the datetime object to "dd/mm/yyyy" format
        formatted_date = date_obj.strftime('%d/%m/%Y')

        return formatted_date
    else:
        return None



In [24]:
from apscheduler.schedulers.background import BackgroundScheduler



def job():
    # Call your function here
    Holding_Percentage()
    company_info()

# Create a scheduler
scheduler = BackgroundScheduler()

# Add the job to the scheduler
scheduler.add_job(job, 'cron', hour=17)  # Set the hour to 17 (5 PM)

# Start the scheduler
scheduler.start()

# Run the job immediately
job()


/usr/local/lib/python3.10/dist-packages/apscheduler/util.py:428: PytzUsageWarning: The localize method is no longer necessary, as this time zone supports the fold attribute (PEP 495). For more details on migrating to a PEP 495-compliant implementation, see https://pytz-deprecation-shim.readthedocs.io/en/latest/migration.html
  return tzinfo.localize(dt)


KeyboardInterrupt: ignored

In [21]:
pip install apscheduler

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 2.4 MB/s eta 0:00:00
